In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())

print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Current device: 0
Device name: NVIDIA GeForce GTX 1650


In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

## 모델이 클래스 특성을 학습하기에 충분한 표본 갯수로 데이터 제거

> Machinery에서 데이터가 30개 이하인 클래스 수: 100
> 
> Assembly에서 데이터가 30개 이하인 클래스 수: 1583
>
> 제거 후, 남은 데이터: 13882, MACHINERY : 62 ASSEMBLY:209

In [3]:
data=pd.read_excel('filtered_dataset_30.xlsx')

In [4]:
print(len(data['Machinery'].unique()),len(data['Assembly'].unique()))

62 209


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13882 entries, 0 to 13881
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        13882 non-null  object 
 1   No.          13882 non-null  int64  
 2   Subject      13872 non-null  object 
 3   Machinery    13882 non-null  object 
 4   Assembly     13882 non-null  object 
 5   청구품목         13882 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Part No.1    13881 non-null  object 
 8   Part No.2    2430 non-null   object 
 9   청구량          13818 non-null  float64
 10  견적           13698 non-null  object 
 11  견적수량         13818 non-null  float64
 12  견적화폐         13818 non-null  object 
 13  견적단가         13882 non-null  float64
 14  발주번호         13882 non-null  object 
 15  발주처          13882 non-null  object 
 16  발주           13882 non-null  object 
 17  발주수량         13818 non-null  float64
 18  발주금액         13818 non-null  float64
 19  D/T 

In [6]:
print(data['청구품목'].unique())

['GE POWER PACK FORK - E7(B)'
 'SAMSON SUPER STRONG DOUBLE BRAID ROPE 1 3/4", 300FT'
 'WIRE ROPE G)6X(S)19 A3 CMP SLPP 28MM X 400M' ... 'BRACKET '
 'WASHER, 10 ' 'COVER,MANIFOLD.EXH ']


### 청구품목 전처리 

1. 텍스트 전처리
2. TF-IDF 기반 강조 (엠퍼사이징)
3. FastText 임베딩
   
### part no.1 전처리

> 콤마 위치에 따른 세부적인 차이가 많은 텍스트이므로 특수기호 및 문자 유지 필요 => 별도 전처리 X

In [29]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)   
    text = re.sub(r'[^\w\s\*/\-\+.,#&]', '', text) 
    text = re.sub(r'\s+', ' ', text)    
    text = re.sub(r'\b(사용금지|사)\b', '', text, flags=re.IGNORECASE)    
    text = text.strip()    
    return text


In [30]:
# 청구품목 클리닝
data['cleaned_item'] = data['청구품목'].apply(preprocess_text)

> 청구품목 데이터에서는 각 단어 의미적 연관성보다 주요단어가 있는 것이므로, 가중치 부여하는 것으로 접근함

In [58]:
# 확인용
claim_items = data['청구품목'].tolist() 

# TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=30)
tfidf_matrix = tfidf.fit_transform(claim_items)

# 중요한 단어 추출
important_words = tfidf.get_feature_names_out()

print(important_words)

['as' 'bearing' 'bolt' 'charges' 'core' 'cover' 'cylinder' 'for' 'fuel'
 'gasket' 'gear' 'gp' 'head' 'hex' 'in' 'kit' 'nut' 'oil' 'plate' 'pump'
 'ring' 'screw' 'seal' 'sensor' 'set' 'shaft' 'spring' 'valve' 'washer'
 'water']


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 2. TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=30) 
tfidf_matrix = tfidf.fit_transform(data['cleaned_item'])

important_words = tfidf.get_feature_names_out()

In [33]:
import nltk
from nltk.corpus import wordnet
import random
# NLTK 증강라이브러리 다운로드
# nltk.download('wordnet')

# 한번에 4개씩 동의어 교체, 새로운 단어 무작위 삽입


def synonym_replacement(text, n=2):
    words = text.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replacements = min(n, len(random_word_list))
    for word in random_word_list[:num_replacements]:
        synonyms = wordnet.synsets(word)
        synonym = random.choice(synonyms).lemmas()[0].name()
        new_words = [synonym if word == w else w for w in new_words]
    return ' '.join(new_words)

def random_insertion(text, n=2):
    words = text.split()
    new_words = words.copy()
    for _ in range(n):
        synonym_word = random.choice(words)
        synonyms = wordnet.synsets(synonym_word)
        if synonyms:
            synonym = random.choice(synonyms).lemmas()[0].name()
            insert_position = random.randint(0, len(new_words))
            new_words.insert(insert_position, synonym)
    return ' '.join(new_words)

# 데이터 증강 함수 통과 후 -> 변형된 텍스트
def augment_text(text):
    augmented_text = synonym_replacement(text, n=4)
    augmented_text = random_insertion(augmented_text, n=4)
    return augmented_text

In [34]:
data['augmented_item'] = data['emphasized_item'].apply(augment_text)

### 발주처 전처리 강화

> 부가단어 (CORPORATION, Corp, CO., Ltd, GmbH, Co., Inc, 주식회사, 상사, 공사, Co.,Ltd, Ltd, Pte Ltd, LLC) 제거

> 핵심 정보(회사명 직접 관련) emphasizing 함 > 증강함수 적용

In [35]:
def clean_supplier_name(name):
    name = name.lower()
    name = re.sub(r'coporation|coropration|coproration|corporration', 'corporation', name)
    name = re.sub(r'\(사용금지\)', '', name)
    name = re.sub(r'u\.s\.a', '_usa', name)
    name = re.sub(r'\.', '', name)
    suffixes = r'(corporation|corp|company|co|incorporated|inc|limited|ltd|상사|공사|엔지니어링|주식회사|주|gmbh|pte ltd|llc)'
    name = re.sub(suffixes, '', name, flags=re.IGNORECASE)
    name = re.sub(r'[^\w\s-]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [36]:
suppliers = [
    'MATSUI(U.S.A) COROPRATION', 'KTI', '대광기업(주)', 'K.TH MARCO',
    'HAEIN Coporation_Cheonan', 'KOREA UCD CO.,LTD.', 'EAST WIND Gmbh', '인스알파',
    'ICON INTERNATIONAL, INC', '한국쉘석유㈜', 'EURO KYTEX ENGINEERING BV', '대동베아링상사',
    'MARINE HYDROTEC CO.,LTD.', '금안상사', 'TEST COMPANY',
    'PORT RELIEF ENGINEERING CO.,LTD.',
    'Caterpillar Marine Asia Pacific Pte Ltd', '(주)혜인',
    'SANWA COMMERCIAL CO.,LTD.', 'yusinHR Co., Ltd.', '(주)선진종합', 'FURUNO',
    'NISSIN REFRIGERATION  ENGINEERIN', '(주)우림공사',
    'HAEIN Coporation_Cheonan(사용금지)', 'KEMEL', 'REXNORD LLC-FALK MARINE GROUP',
    '유신에이치알(사용금지)', 'GEA KOREA LTD', '주안에너지㈜', 'SUNJIN ETECH Co.,Ltd.',
    '디에스알제강주식회사', '(주)한국에프에이디', 'ALBERT GMBH', 'Wartsila Korea Ltd.',
    '(주)선진엔텍(사용금지)', 'PIRIOU NAVAL', '(주)프러스엔지니어링', 'Taeyoung Enterprise',
    'SHINA', 'INS ALFA', 'KEMEL(KOMARINE)', '누리엔지니어링', 'RNK TECH CO.,LTD',
    'OS SYSTEM CO.,LTD', '씨코리아엔지니어링(주)', '(주)두원알앤에이', '합동듸젤사', '하이에어코리아(주)',
    'DESMI PUMPING TECHNOLOGY(SUZHOU) CO.,LTD', '한국마이콤',
    'HUMAN & ENGINEERING CO.,LTD'
]

cleaned_suppliers = [clean_supplier_name(supplier) for supplier in suppliers]
print(cleaned_suppliers)


['matsui_usa', 'kti', '대광기업', 'kth mar', 'haein _cheonan', 'korea ucd', 'east wind', '인스알파', 'in international', '한국쉘석유', 'euro kytex engineering bv', '대동베아링', 'marine hydrotec', '금안', 'test', 'port relief engineering', 'caterpillar marine asia pacific', '혜인', 'sanwa mmercial', 'yusinhr', '선진종합', 'furuno', 'nissin refrigeration engineerin', '우림', 'haein _cheonan', 'kemel', 'rexnord -falk marine group', '유신에이치알', 'gea korea', '안에너지', 'sunjin etech', '디에스알제강', '한국에프에이디', 'albert', 'wartsila korea', '선진엔텍', 'piriou naval', '프러스', 'taeyoung enterprise', 'shina', 'ins alfa', 'kemelkomarine', '누리', 'rnk tech', 'os system', '씨코리아', '두원알앤에이', '합동듸젤사', '하이에어코리아', 'desmi pumping technologysuzhou', '한국마이콤', 'human engineering']


In [37]:
#  HAEIN Corporation => HAEIN
def extract_important_part(name):
    if re.search(r'[가-힣]', name):
        name = re.sub(r'(기업|상사|종합|공사)', '', name)
        important_part = name.split()[0]  # 첫 단어 추출
    else:
        # 영문 이름의 경우 첫 번째 단어만 추출
        important_part = name.split()[0]
    
    return important_part

# 한번더 반복 HAEIN HAEIN Corporation
def emphasize_supplier_name(name):
    important_part = extract_important_part(name)
    emphasized_name = f"{important_part} {important_part} {name}"  # 중요한 부분을 반복
    return emphasized_name

In [38]:
suppliers = ['MATSUI(U.S.A) COROPRATION', 'taeyoung enterprise','HAEIN Coporation_Cheonan(사용금지)']
cleaned_suppliers = [clean_supplier_name(supplier) for supplier in suppliers]  # 전처리
emphasized_suppliers = [emphasize_supplier_name(supplier) for supplier in cleaned_suppliers]  # 강조

print(emphasized_suppliers)

['matsui_usa matsui_usa matsui_usa', 'taeyoung taeyoung taeyoung enterprise', 'haein haein haein _cheonan']


### 결합

> 청구품목 데이터 증강 수행 후, 원본 발주처 및 part no.1과 일대일로 결합

In [40]:
# 청구품목 = TF-IDF 강조 적용 + 증강된 것
# data['augmented_item'] = data['emphasized_item'].apply(augment_text)

#파트넘버(별도 전처리X)
data['Part No.1'] = data['Part No.1'].astype(str)

# 발주처
data['cleaned_supplier'] = data['발주처'].apply(clean_supplier_name)
data['emphasized_supplier'] = data['cleaned_supplier'].apply(emphasize_supplier_name)


In [41]:

augmented_data = []

for _, row in data.iterrows():
    # 청구품목을 10개씩 증강하고 그 수만큼 발주처와 Part No.1을 복제
    augmented_item_list = [augment_text(row['augmented_item']) for _ in range(10)]
    for augmented_item in augmented_item_list:
        augmented_data.append([augmented_item, row['Part No.1'], row['emphasized_supplier']])

# 증강된 데이터를 DataFrame으로 변환
augmented_df = pd.DataFrame(augmented_data, columns=['augmented_item', 'Part No.1', 'emphasized_supplier'])

# 원본 데이터와 결합 (원본 데이터도 포함)
combined_df = pd.concat([data[['augmented_item', 'Part No.1', 'emphasized_supplier']], augmented_df], ignore_index=True)

# 최종 combined_text 결합
combined_df['combined_text'] = combined_df['augmented_item'].fillna('') + " " + combined_df['Part No.1'].fillna('') + " " + combined_df['emphasized_supplier'].fillna('')

In [44]:
augmented_df = pd.DataFrame(augmented_data, columns=['augmented_item', 'Part No.1', 'emphasized_supplier'])
augmented_df.head()

,augmented_item,Part No.1,emphasized_supplier
0,Gaea Gaea might Gaea might Gaea pack fork - e7...,40028340,matsui_usa matsui_usa matsui_usa
1,Gaea power Gaea power pack Gaea fork - exponen...,40028340,matsui_usa matsui_usa matsui_usa
2,Gaea power Gaea power battalion pitchfork - e7...,40028340,matsui_usa matsui_usa matsui_usa
3,Gaea world_power Gaea world_power tamp_down Ga...,40028340,matsui_usa matsui_usa matsui_usa
4,Gaea Gaea Gaea might Gaea might pack fork - e7...,40028340,matsui_usa matsui_usa matsui_usa


## fasttext 임베딩
> 전처리에서 가중치 부여된 데이터로 fasttext 학습했으므로 임베딩에서는 가중처리 방식 선택하지 않음

> 평균 시퀀스를 기준 패딩처리함

In [18]:
from gensim.models import FastText

sentences = [text.split() for text in data['combined_text'].tolist()]

# FastText 모델, 사전, 트레인
ft_model = FastText(vector_size=120, window=5, min_count=1, min_n=2, max_n=7, sg=1)
ft_model.build_vocab(sentences)
ft_model.train(sentences, total_examples=len(sentences), epochs=30)


(3371424, 5151930)

In [19]:
sequence_lengths = [len(text.split()) for text in data['combined_text']]
average_length = sum(sequence_lengths) / len(sequence_lengths)
print(f"평균 시퀀스 길이: {average_length}")

평균 시퀀스 길이: 12.370767900878835


In [20]:
import torch

# 패딩 함수
def pad_embeddings(embeddings, target_length, model):
    current_length = embeddings.size(0)
    if current_length < target_length:
        padding = torch.zeros(target_length - current_length, model.vector_size)
        embeddings = torch.cat([embeddings, padding], dim=0)
    elif current_length > target_length:
        embeddings = embeddings[:target_length]
    return embeddings

# 패딩처리하고 결합, 시퀀스 길이에 맞춰 타겟 길이 조정함
target_length = 16
combined_embeddings = []

for _, row in data.iterrows():
    # **증강된 청구품목**을 반영하여 임베딩 생성
    item_embedding = torch.tensor([ft_model.wv[word] for word in row['augmented_item'].split() if word in ft_model.wv])
    part_embedding = torch.tensor([ft_model.wv[word] for word in row['Part No.1'].split() if word in ft_model.wv])  # 전처리X
    supplier_embedding = torch.tensor([ft_model.wv[word] for word in row['emphasized_supplier'].split() if word in ft_model.wv])

    # 각각의 임베딩을 패딩하여 고정 크기로 맞춤
    item_embedding_padded = pad_embeddings(item_embedding, target_length, ft_model)
    part_embedding_padded = pad_embeddings(part_embedding, target_length, ft_model)
    supplier_embedding_padded = pad_embeddings(supplier_embedding, target_length, ft_model)

    # 패딩된 임베딩들을 결합
    combined_embedding = torch.cat([item_embedding_padded, part_embedding_padded, supplier_embedding_padded], dim=0)
    combined_embeddings.append(combined_embedding)

# 최종 텐서로 변환
combined_embeddings_tensor = torch.stack(combined_embeddings)

X = combined_embeddings_tensor
print(X.shape)

C:\Users\User\AppData\Local\Temp\ipykernel_13576\2339533233.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  item_embedding = torch.tensor([ft_model.wv[word] for word in row['augmented_item'].split() if word in ft_model.wv])


torch.Size([13882, 48, 120])


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


# 정수형 레이블이어야 분류할 수 있음
machinery_labels = data['Machinery'].values
assembly_labels = data['Assembly'].values

label_encoder_machinery = LabelEncoder()
y_machinery = label_encoder_machinery.fit_transform(machinery_labels)

label_encoder_assembly = LabelEncoder()
y_assembly = label_encoder_assembly.fit_transform(assembly_labels)

# 스케일러 적용
X = combined_embeddings_tensor
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

# Train-Test Split 그대로 유지
X_train_val, X_test, y_train_val_machinery, y_test_machinery, y_train_val_assembly, y_test_assembly = train_test_split(
    X_scaled, y_machinery, y_assembly, test_size=0.2, random_state=42, stratify=y_machinery
)

X_train, X_val, y_train_machinery, y_val_machinery, y_train_assembly, y_val_assembly = train_test_split(
    X_train_val, y_train_val_machinery, y_train_val_assembly, test_size=0.2, random_state=42, stratify=y_train_val_machinery
)

# CNN 모델용 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train_machinery, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_machinery, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_machinery, dtype=torch.long)

# XGBoost 모델용 NumPy 배열 준비 (이미 X_scaled는 NumPy 배열)
X_train_np = X_train
X_val_np = X_val
X_test_np = X_test

y_train_np = y_train_machinery
y_val_np = y_val_machinery
y_test_np = y_test_machinery

### CNN

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization, GlobalMaxPooling1D, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# 개선된 CNN 모델
machinery_model = Sequential([
    Conv1D(256, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    
    Conv1D(256, kernel_size=5, activation='relu'),  # 추가 Conv1D 레이어
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    
    GlobalMaxPooling1D(),  
    
    Dense(128, kernel_regularizer=l2(0.01)), 
    LeakyReLU(alpha=0.01),
    BatchNormalization(),
    Dropout(0.4),  # Dropout 비율 조정
    
    Dense(64, kernel_regularizer=l2(0.001)),  
    LeakyReLU(alpha=0.01),
    Dropout(0.3),  # Dropout 추가
    
    Dense(62, activation='softmax')  # Machinery 클래스 수
])

# 옵티마이저와 학습률 조정
optimizer = Adam(learning_rate=0.0001)
machinery_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# 모델 학습
history = machinery_model.fit(X_train, y_train_machinery, epochs=50, batch_size=32, validation_data=(X_val, y_val_machinery), callbacks=[early_stopping, reduce_lr])

C:\Users\User\anaconda3\envs\ship\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\User\anaconda3\envs\ship\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.2757 - loss: 5.0515 - val_accuracy: 0.0662 - val_loss: 5.1046 - learning_rate: 1.0000e-04
Epoch 2/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5335 - loss: 3.7729 - val_accuracy: 0.5700 - val_loss: 3.3911 - learning_rate: 1.0000e-04
Epoch 3/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5605 - loss: 3.3634 - val_accuracy: 0.6119 - val_loss: 2.9379 - learning_rate: 1.0000e-04
Epoch 4/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5788 - loss: 3.1024 - val_accuracy: 0.6177 - val_loss: 2.8218 - learning_rate: 1.0000e-04
Epoch 5/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5937 - loss: 2.8984 - val_accuracy: 0.6326 - val_loss: 2.6309 - learning_rate: 1.0000e-04
Epoch 6/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6164 - loss: 2.7218 - val_accuracy: 0.6236 - val_loss: 2.5297 - learning_rate: 1.0000e-04
Epoch 7/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - acc

In [27]:
from xgboost import XGBClassifier

machinery_optimized_model = XGBClassifier(
    objective='multi:softmax',
    num_class=62,
    learning_rate=0.0746,
    max_depth=10,
    n_estimators=148,
    subsample=1.0,
    colsample_bytree=0.6804,
    reg_lambda=1,
    random_state=42,
    verbosity=1
)


In [28]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization, GlobalMaxPooling1D, LeakyReLU
from sklearn.pipeline import Pipeline

# 1. CNN 모델 정의 (변환기 클래스 사용)
class CNNTransformer(TransformerMixin):
    def __init__(self, input_shape, epochs=100, batch_size=32):
        self.model = self.build_cnn_model(input_shape)
        self.epochs = epochs
        self.batch_size = batch_size
        
    def build_cnn_model(self, input_shape):
        model = Sequential([
            Conv1D(256, kernel_size=5, activation='relu', input_shape=input_shape),
            MaxPooling1D(pool_size=2),
            BatchNormalization(),
            Conv1D(256, kernel_size=5, activation='relu'),
            MaxPooling1D(pool_size=2),
            BatchNormalization(),
            GlobalMaxPooling1D(),
            Dense(128, kernel_regularizer=l2(0.01)),
            LeakyReLU(alpha=0.01),
            BatchNormalization(),
            Dropout(0.4),
            Dense(64, kernel_regularizer=l2(0.001)),
            LeakyReLU(alpha=0.01),
            Dropout(0.3),
            Dense(62, activation='softmax')  # Machinery 클래스 수
        ])
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self

    def transform(self, X):
        return self.model.predict(X)

# 2. Stacking 모델 정의
cnn_transformer = CNNTransformer(input_shape=(X_train.shape[1], X_train.shape[2]))

stacked_model = StackingClassifier(
    estimators=[
        ('cnn', Pipeline([('cnn_transformer', cnn_transformer)])),  # CNN 모델 변환 후 입력
        ('xgb', machinery_optimized_model)  # XGBoost 모델
    ],
    final_estimator=LogisticRegression()  # 최종 분류기로 Logistic Regression 사용
)

# 3. Stacking 모델 학습
stacked_model.fit(X_train, y_train_machinery)

# 예측 및 평가
y_pred = stacked_model.predict(X_test)
accuracy = accuracy_score(y_test_machinery, y_pred)
print(f"Stacked Model Accuracy: {accuracy:.4f}")

C:\Users\User\anaconda3\envs\ship\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: Please reshape the input data into 2-dimensional matrix.